# Modify Background Database

In this project, certain processes based on exisiting processes contained in the background databases must be added to their respective background database. These processes are based on:

- diesel, burned in building machine - GLO; from `EI_DB_NAME`;
- transport, freight, lorry 7.5-16 metric ton, EURO5 - RER; from `EI_DB_NAME`;
- transport, passenger car, medium size, petrol, EURO 5 - RER; from `EI_DB_NAME`;
- basalt quarry operation - RER; from `EI_DB_NAME`;
- gravel production, crushed - RoW; from `EI_DB_NAME`;
- cellulose fibre production - RoW from `EI_DB_NAME`.


In [1]:
from pprint import pprint

import bw2data as bd
import bw2io as bi
from bw2data.query import Filter, Query
from tabulate import tabulate

In [2]:
# Importing the variables with the project name and background db
from project_details import EI_DB_NAME, PROJECT_NAME

In [3]:
# Set project
bd.projects.set_current(PROJECT_NAME)
# Check databases included in project
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff

In [4]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert EI_DB_NAME in bd.databases

## Adding modified diesel processes
First, two processes based on the ecoinvent process "diesel, burned in building machine - GLO" must be added. These processes correspond to the diesel consumption attributed to the machinery use to lay and mill asphalt during the construction (A5) and EOL (C1) life cycle phases of our system under analysis. Therefore, we will create two processes named as follows:

- A5, diesel, burned in building machine - GLO
- C1, diesel, burned in building machine - GLO

The emissions of these processes are modified to comply with the specifications of the [NL-PCR](https://www.bouwendnederland.nl/media/12923/pcr-asfalt-v20.pdf).

The emissions that we are modifying and their respective new values are:

| Name                                               | A5 (kg)               | C1 (kg)               |
|----------------------------------------------------|-----------------------|-----------------------|
| Ammonia                                            | 2.33222e-07           | 2.33222e-07           |
| Carbon dioxide, fossil                            | 7.44086e-05           | 7.44086e-05           |
| Carbon monoxide, fossil                           | 3.19444e-08           | 1.00901e-08           |
| Dinitrogen monoxide                               | 4.99833e-10           | 4.99833e-10           |
| Nitrogen oxides                                   | 7.67056e-08           | 3.83334e-08           |
| NMVOC, non-methane volatile organic compounds     | 8.91294e-10           | 4.08334e-10           |
| Particulate Matter, < 2.5 um                      | 1.29832e-09           | 1.63390e-09           |
| Particulate Matter, > 10 um                       | 6.83056e-11           | 3.22335e-09           |
| Particulate Matter, > 2.5 um and < 10um           | 1.23587e-09           | 1.89703e-09           |
| Sulfur dioxide                                    | 4.66667e-10           | 4.66667e-10           |

Furthermore, the diesel exchange *market group for diesel - GLO* must be replaced by *market for diesel, low-sulfur - Europe without Switzerland* in both new processes.

In [5]:
EI_DB = bd.Database(EI_DB_NAME)

In [6]:
# Delete the diesel a5 and c1 in case we ran this notebook multiple times
res = EI_DB.search("A5, diesel, burned in building machine")
for r in res:
    print(r)
    r.delete()
res = EI_DB.search("C1, diesel, burned in building machine")
for r in res:
    print(r)
    r.delete()

'A5, diesel, burned in building machine' (megajoule, GLO, None)
'C1, diesel, burned in building machine' (megajoule, GLO, None)


In [7]:
eidb_data = bd.Database(EI_DB_NAME).load()

### copy original processes

In [8]:
# Search for original diesel process in background database
q = Query()
filter_name = Filter("name", "is", "diesel, burned in building machine")
filter_location = Filter("location", "is", "GLO")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    diesel = bd.get_activity(list(res).pop())
diesel

'diesel, burned in building machine' (megajoule, GLO, None)

In [9]:
# Look at exhanges characterizing activity
rows = []
for exc in diesel.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"diesel, burned in building machine",megajoule,1,"'diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,building machine,unit,1.34e-07,"'market for building machine' (unit, GLO, None)"
technosphere,diesel,kilogram,0.0234,"'market group for diesel' (kilogram, GLO, None)"
technosphere,lubricating oil,kilogram,8.9436e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,lubricating oil,kilogram,0.000424564,"'market for lubricating oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000369419,"'market for waste mineral oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000141088,"'market for waste mineral oil' (kilogram, Europe without Switzerland, None)"
technosphere,waste mineral oil,kilogram,-3.49279e-06,"'market for waste mineral oil' (kilogram, CH, None)"
biosphere,Ammonia,kilogram,4.67e-07,"'Ammonia' (kilogram, None, ('air',))"
biosphere,Benzo(a)pyrene,kilogram,7.01e-10,"'Benzo(a)pyrene' (kilogram, None, ('air',))"
biosphere,Cadmium II,kilogram,2.34e-10,"'Cadmium II' (kilogram, None, ('air',))"


In [10]:
# Copy existing diesel process
diesel_A5 = diesel.copy(name="A5, diesel, burned in building machine")
diesel_C1 = diesel.copy(name="C1, diesel, burned in building machine")

In [11]:
# Check that new processes are saved to background database
EI_DB.search("diesel, burned in building machine")

['market for diesel, burned in building machine' (megajoule, GLO, None),
 'diesel, burned in building machine' (megajoule, GLO, None),
 'A5, diesel, burned in building machine' (megajoule, GLO, None),
 'C1, diesel, burned in building machine' (megajoule, GLO, None),
 'propane, burned in building machine' (megajoule, GLO, None),
 'market for propane, burned in building machine' (megajoule, GLO, None),
 'market for diesel, burned in diesel-electric generating set, 10MW' (megajoule, GLO, None)]

In [12]:
# Look at exhanges characterizing activity
rows = []
for exc in diesel_A5.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"diesel, burned in building machine",megajoule,1,"'A5, diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,building machine,unit,1.34e-07,"'market for building machine' (unit, GLO, None)"
technosphere,diesel,kilogram,0.0234,"'market group for diesel' (kilogram, GLO, None)"
technosphere,lubricating oil,kilogram,8.9436e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,lubricating oil,kilogram,0.000424564,"'market for lubricating oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000369419,"'market for waste mineral oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000141088,"'market for waste mineral oil' (kilogram, Europe without Switzerland, None)"
technosphere,waste mineral oil,kilogram,-3.49279e-06,"'market for waste mineral oil' (kilogram, CH, None)"
biosphere,Ammonia,kilogram,4.67e-07,"'Ammonia' (kilogram, None, ('air',))"
biosphere,Benzo(a)pyrene,kilogram,7.01e-10,"'Benzo(a)pyrene' (kilogram, None, ('air',))"
biosphere,Cadmium II,kilogram,2.34e-10,"'Cadmium II' (kilogram, None, ('air',))"


In [13]:
# detail look to exchanges of interest
[
    exc for exc in diesel_A5.exchanges() if exc.get("name", "") == "Ammonia"
].pop().as_dict()  # Ammonia

{'flow': '87883a4e-1e3e-4c9d-90c0-f1bea36f8014',
 'type': 'biosphere',
 'name': 'Ammonia',
 'classifications': {'CPC': []},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'unit': 'kilogram',
 'amount': 4.67e-07,
 'pedigree': {'reliability': 1,
  'completeness': 5,
  'temporal correlation': 5,
  'geographical correlation': 3,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': -14.576936579277515,
 'scale': 0.3132091952673165,
 'scale without pedigree': 0.22360679774997896

### define new values for the exchanges of the new activities

In [14]:
# Create dictionaries with new emissions for each activity
A5_emissions = [
    {"name": "Ammonia", "amount": 2.33222e-07},
    {"name": "Carbon dioxide, fossil", "amount": 7.44086e-05},
    {"name": "Carbon monoxide, fossil", "amount": 3.19444e-08},
    {"name": "Dinitrogen monoxide", "amount": 4.99833e-10},
    {"name": "Nitrogen oxides", "amount": 7.67056e-08},
    {"name": "NMVOC, non-methane volatile organic compounds", "amount": 8.91294e-10},
    {"name": "Particulate Matter, < 2.5 um", "amount": 1.29832e-09},
    {"name": "Particulate Matter, > 10 um", "amount": 6.83056e-11},
    {"name": "Particulate Matter, > 2.5 um and < 10um", "amount": 1.23587e-09},
    {"name": "Sulfur dioxide", "amount": 4.66667e-10},
]

C1_emissions = [
    {"name": "Ammonia", "amount": 2.33222e-07},
    {"name": "Carbon dioxide, fossil", "amount": 7.44086e-05},
    {"name": "Carbon monoxide, fossil", "amount": 1.00901e-08},
    {"name": "Dinitrogen monoxide", "amount": 4.99833e-10},
    {"name": "Nitrogen oxides", "amount": 3.83334e-08},
    {"name": "NMVOC, non-methane volatile organic compounds", "amount": 4.08334e-10},
    {"name": "Particulates, <2.5 µm", "amount": 1.63390e-09},
    {"name": "Particulates, >2.5 µm and <10 µm", "amount": 3.22335e-09},
    {"name": "Particulates, >10 µm", "amount": 1.89703e-09},
    {"name": "Sulphur dioxide", "amount": 4.66667e-10},
]

### modify the emissions of the new activities

In [15]:
def modify_emission(exc, new_emissions):
    """
    Modify the amount of the exchange, based on the new_emissions list.

    the new emissions list contains dictionaries.
    Each dictionnary specifies an emission name and the amount to use.
    """
    new_emission_names = [emission["name"] for emission in new_emissions]
    # only apply a change if the exchange name is in the new_emissions name
    if exc.get("name", "") in new_emission_names:
        new_emission = [
            emission for emission in new_emissions if emission["name"] == exc["name"]
        ][0]
        # print(f"Modifying {exc['name']} with {new_emission}")
        exc["amount"] = new_emission["amount"]
        exc.save()

In [16]:
# Modify exchanges in diesel_A5
for exc in diesel_A5.exchanges():
    modify_emission(exc, A5_emissions)

In [17]:
# Modify exchanges in diesel_C1
for exc in diesel_C1.exchanges():
    modify_emission(exc, C1_emissions)

Verification: were the exchange amounts indeed updated?

In [18]:
# Look at exhanges characterizing activity diesel_A5
rows = []
for exc in diesel_A5.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"diesel, burned in building machine",megajoule,1,"'A5, diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,building machine,unit,1.34e-07,"'market for building machine' (unit, GLO, None)"
technosphere,diesel,kilogram,0.0234,"'market group for diesel' (kilogram, GLO, None)"
technosphere,lubricating oil,kilogram,8.9436e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,lubricating oil,kilogram,0.000424564,"'market for lubricating oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000369419,"'market for waste mineral oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000141088,"'market for waste mineral oil' (kilogram, Europe without Switzerland, None)"
technosphere,waste mineral oil,kilogram,-3.49279e-06,"'market for waste mineral oil' (kilogram, CH, None)"
biosphere,Ammonia,kilogram,2.33222e-07,"'Ammonia' (kilogram, None, ('air',))"
biosphere,Benzo(a)pyrene,kilogram,7.01e-10,"'Benzo(a)pyrene' (kilogram, None, ('air',))"
biosphere,Cadmium II,kilogram,2.34e-10,"'Cadmium II' (kilogram, None, ('air',))"


### Modify the source of the makert group for diesel
Now we change the diesel exchange from *market group for diesel - GLO* to *market for diesel, low-sulfur - Europe without Switzerland* in the new activities.

In [19]:
# Store original diesel exchange as a variable
diesel_og = [
    exc for exc in diesel_A5.exchanges() if "diesel" in exc.get("name", "")
].pop()  # Market group for diesel
diesel_og

Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>

In [20]:
# Take a closer look to diesel_og
diesel_og.as_dict()

{'flow': '291fc06d-1b3e-4077-aabb-346b588ed24b',
 'type': 'technosphere',
 'name': 'diesel',
 'classifications': {'CPC': ['33360: Gas oil']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865,
   'comment': 'C-content of diesel is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.14)',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.8,
   'comment': 'Based on literature, page 16, Report6, part 4',
   'unit': 'MJ'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '4df5d193-687d-534a-b8e5-9f80e6a93b2d',
 'unit': 'kilogram',
 'comment': 'EcoSpold

In [21]:
# find replacement exchange for diesel_og: market for diesel, low-sulfur - Europe without Switzerland
q = Query()
filter_name = Filter("name", "is", "market for diesel, low-sulfur")
filter_location = Filter("location", "is", "Europe without Switzerland")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    diesel_ls = bd.get_activity(list(res).pop())
# store replacement activity as variable
diesel_ls

'market for diesel, low-sulfur' (kilogram, Europe without Switzerland, None)

In [22]:
# Take a closer look to diesel_ls
diesel_ls.as_dict()

{'comment': "This is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nThis is the market for  'diesel, low-sulfur', in the geography of Europe without Switzerland.\nInventory for the distribution of petroleum product to the final consumer (household, car, power plant, etc.) including all necessary transports.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinven

In [23]:
# add diesel_ls to diesel_x exchanges based on the diesel_og exchange parameters
diesel_ls_A5 = diesel_A5.new_exchange(
    input=diesel_ls.key,
    amount=diesel_og.amount,
    unit=diesel_og.unit,
    type="technosphere",
    name="diesel",
)
diesel_ls_C1 = diesel_C1.new_exchange(
    input=diesel_ls.key,
    amount=diesel_og.amount,
    unit=diesel_og.unit,
    type="technosphere",
    name="diesel",
)
diesel_ls_A5.save()
diesel_ls_C1.save()
# The calls below might not be necessary
diesel_A5.save()
diesel_C1.save()

In [24]:
# Look at exhanges characterizing activity diesel_A5
rows = []
for exc in diesel_A5.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"diesel, burned in building machine",megajoule,1,"'A5, diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,building machine,unit,1.34e-07,"'market for building machine' (unit, GLO, None)"
technosphere,diesel,kilogram,0.0234,"'market group for diesel' (kilogram, GLO, None)"
technosphere,lubricating oil,kilogram,8.9436e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,lubricating oil,kilogram,0.000424564,"'market for lubricating oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000369419,"'market for waste mineral oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000141088,"'market for waste mineral oil' (kilogram, Europe without Switzerland, None)"
technosphere,waste mineral oil,kilogram,-3.49279e-06,"'market for waste mineral oil' (kilogram, CH, None)"
biosphere,Ammonia,kilogram,2.33222e-07,"'Ammonia' (kilogram, None, ('air',))"
biosphere,Benzo(a)pyrene,kilogram,7.01e-10,"'Benzo(a)pyrene' (kilogram, None, ('air',))"
biosphere,Cadmium II,kilogram,2.34e-10,"'Cadmium II' (kilogram, None, ('air',))"


In [25]:
# Delete diesel exchange
victim_exc = None
for exch in diesel_A5.technosphere():
    exch_activity = bd.get_activity(exch["input"])
    if exch_activity.get("name") == "market group for diesel":
        victim_exc = exch
if victim_exc:
    print(f"Deleteing exchange {victim_exc}")
    victim_exc.delete()

victim_exc = None
for exch in diesel_C1.technosphere():
    exch_activity = bd.get_activity(exch["input"])
    if exch_activity.get("name") == "market group for diesel":
        victim_exc = exch
if victim_exc:
    print(f"Deleteing exchange {victim_exc}")
    victim_exc.delete()

Deleteing exchange Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Deleteing exchange Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'C1, diesel, burned in building machine' (megajoule, GLO, None)>


In [26]:
# Verify characterizing activity diesel_A5
rows = []
for exc in diesel_A5.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"diesel, burned in building machine",megajoule,1,"'A5, diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,building machine,unit,1.34e-07,"'market for building machine' (unit, GLO, None)"
technosphere,lubricating oil,kilogram,8.9436e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,lubricating oil,kilogram,0.000424564,"'market for lubricating oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000369419,"'market for waste mineral oil' (kilogram, RoW, None)"
technosphere,waste mineral oil,kilogram,-0.000141088,"'market for waste mineral oil' (kilogram, Europe without Switzerland, None)"
technosphere,waste mineral oil,kilogram,-3.49279e-06,"'market for waste mineral oil' (kilogram, CH, None)"
biosphere,Ammonia,kilogram,2.33222e-07,"'Ammonia' (kilogram, None, ('air',))"
biosphere,Benzo(a)pyrene,kilogram,7.01e-10,"'Benzo(a)pyrene' (kilogram, None, ('air',))"
biosphere,Cadmium II,kilogram,2.34e-10,"'Cadmium II' (kilogram, None, ('air',))"
biosphere,"Carbon dioxide, fossil",kilogram,7.44086e-05,"'Carbon dioxide, fossil' (kilogram, None, ('air',))"


In [27]:
# Verify diesel_ls exchange
for exc in diesel_A5.technosphere():
    exc_activity = bd.get_activity(exc["input"])
    if exc_activity.get("name") == "market for diesel, low-sulfur":
        pprint(exc)

Exchange: 0.0234 kilogram 'market for diesel, low-sulfur' (kilogram, Europe without Switzerland, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>


## Adding modified transport processes

Two new transport processes to model the extra fuel consumption due to pavement deterioration in the use phase (B) must be added to the background. These processes are based on the following ecoinvent processes:

The new processes are modified to only include the exchanges related to fuel (diesel and petrol). The name of the new processes is:

- transport, freight, lorry 7.5-16 metric ton, EURO5 - RER
- transport, passenger car, medium size, petrol, EURO 5 - RER

The new processes are modified to only include the technosphere exchanges related to fuel (diesel and petrol). The names of the new processeses are:
- B, transport, freight, lorry 7.5-16 metric ton, EURO5 - RER
- B, transport, passenger car, medium size, petrol, EURO 5 - RER

In [28]:
# Delete the diesel a5 and c1 in case we ran this notebook multiple times
res = EI_DB.search("B, transport, freight, lorry 7.5-16 metric ton, EURO5")
for r in res:
    print(r)
    r.delete()
res = EI_DB.search("B, transport, passenger car, medium size, petrol, EURO 5")
for r in res:
    print(r)
    r.delete()

'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)
'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)


### copy original processes

In [29]:
# Search for transport processes in background database
q = Query()
filter_name = Filter("name", "is", "transport, freight, lorry 7.5-16 metric ton, EURO5")
filter_location = Filter("location", "is", "RER")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    lorry = bd.get_activity(list(res).pop())
print(lorry)

'transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)


In [30]:
lorry_B = lorry.copy(name="B, transport, freight, lorry 7.5-16 metric ton, EURO5")

In [31]:
# Search for transport processes in background database
q = Query()
filter_name = Filter(
    "name", "is", "transport, passenger car, medium size, petrol, EURO 5"
)
filter_location = Filter("location", "is", "RER")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    car = bd.get_activity(list(res).pop())
print(car)

'transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)


In [32]:
car_B = car.copy(name="B, transport, passenger car, medium size, petrol, EURO 5")

In [33]:
# Verify that the new transport processes were recorded
EI_DB.search("B, transport,")

['B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None),
 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None),
 'market for cement, CEM III/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM III/B' (kilogram, CH, None),
 'market for cement, CEM V/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM II/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM IV/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM IV/B' (kilogram, CH, None),
 'market for cement, CEM II/B' (kilogram, CH, None),
 'market for cement, CEM V/B' (kilogram, CH, None),
 'market for cement, CEM III/B' (kilogram, RoW, None),
 'market for cement, CEM V/B' (kilogram, RoW, None),
 'market for cement, CEM II/B' (kilogram, RoW, None),
 'market for cement, CEM IV/B' (kilogram, RoW, None),
 'market for cement, CEM II/B-L' (kilogram, ZA, None),
 'market for cement, CEM II/B-S' (kilogr

First, let's update B, transport, freight, lorry 7.5-16 metric ton, EURO5

In [34]:
# Look at technosphere exhanges characterizing activity lorry_B
rows = []
for exc in lorry_B.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"transport, freight, lorry 7.5-16 metric ton, EURO5",ton kilometer,1,"'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)"
technosphere,"brake wear emissions, lorry",kilogram,-2.29493e-05,"'market for brake wear emissions, lorry' (kilogram, GLO, None)"
technosphere,"diesel, low-sulfur",kilogram,0.0473021,"'market group for diesel, low-sulfur' (kilogram, RER, None)"
technosphere,"lorry, 16 metric ton",unit,5.63e-07,"'market for lorry, 16 metric ton' (unit, RER, None)"
technosphere,"maintenance, lorry 16 metric ton",unit,5.63e-07,"'market for maintenance, lorry 16 metric ton' (unit, RER, None)"
technosphere,road,meter-year,0.00109,"'market for road' (meter-year, GLO, None)"
technosphere,road maintenance,meter-year,0.000392949,"'market for road maintenance' (meter-year, RER, None)"
technosphere,"road wear emissions, lorry",kilogram,-1.97596e-05,"'market for road wear emissions, lorry' (kilogram, GLO, None)"
technosphere,"tyre wear emissions, lorry",kilogram,-0.000227376,"'market for tyre wear emissions, lorry' (kilogram, GLO, None)"
biosphere,Acetaldehyde,kilogram,2.27273e-07,"'Acetaldehyde' (kilogram, None, ('air',))"
biosphere,Acrolein,kilogram,8.80246e-08,"'Acrolein' (kilogram, None, ('air',))"


In [35]:
# detail look to exchanges of interest
[
    exc for exc in lorry_B.technosphere() if exc.get("name", "") == "diesel, low-sulfur"
].pop().as_dict()  # market group for diesel, low-sulfur

{'flow': 'b80eb2d5-1256-41bb-a61d-83252d0dfd4d',
 'type': 'technosphere',
 'name': 'diesel, low-sulfur',
 'classifications': {'CPC': ['33360: Gas oil']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865,
   'comment': 'C-content of diesel low-sulfur is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.14)',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.8,
   'comment': 'Based on literature, page 16, Report6, part 4',
   'unit': 'MJ'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '427d868d-e90c-5da5-bde4-ff61ebd30a6f',
 'unit': 'kilogram'

### Remove all exchanges from lorry_B and car_B, except diesel and petrol low-sulfur

In [36]:
# Iterate over exchanges in lorry_B
for exc in lorry_B.technosphere():
    if exc["name"] != "diesel, low-sulfur":
        # Delete exchange
        exc.delete()

Let's verify the exchanges for the activity  now

In [37]:
# Look at exhanges characterizing activity lorry_B
rows = []
for exc in lorry_B.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"diesel, low-sulfur",kilogram,0.0473021,"'market group for diesel, low-sulfur' (kilogram, RER, None)"


Let's repeat for B, transport, passenger car, medium size, petrol, EURO 5

In [38]:
# Look at technosphere exhanges characterizing activity car_B
rows = []
for exc in car_B.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"brake wear emissions, passenger car",kilogram,-7.55254e-06,"'market for brake wear emissions, passenger car' (kilogram, GLO, None)"
technosphere,passenger car maintenance,unit,8.60215e-06,"'maintenance, passenger car' (unit, RER, None)"
technosphere,"passenger car, petrol/natural gas",kilogram,0.0106667,"'market for passenger car, petrol/natural gas' (kilogram, GLO, None)"
technosphere,"petrol, low-sulfur",kilogram,0.0609059,"'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None)"
technosphere,"petrol, low-sulfur",kilogram,0.00116086,"'market for petrol, low-sulfur' (kilogram, CH, None)"
technosphere,road,meter-year,0.000911396,"'market for road' (meter-year, GLO, None)"
technosphere,road maintenance,meter-year,0.000265427,"'market for road maintenance' (meter-year, RER, None)"
technosphere,"road wear emissions, passenger car",kilogram,-1.66156e-05,"'market for road wear emissions, passenger car' (kilogram, GLO, None)"
technosphere,"tyre wear emissions, passenger car",kilogram,-9.71647e-05,"'market for tyre wear emissions, passenger car' (kilogram, GLO, None)"


In [39]:
# detail look to exchanges of interest
[exc for exc in car_B.technosphere() if "petrol, low-sulfur" in exc.get("name", "")]
# market for petrol, low-sulfur - Europe without Szwitzerland
# market for petrol, low-sulfur - CH

[Exchange: 0.06090593821630414 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>,
 Exchange: 0.001160855383695865 kilogram 'market for petrol, low-sulfur' (kilogram, CH, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>]

In [40]:
# Iterate over exchanges in car_B
for exc in car_B.technosphere():
    if exc["name"] != "petrol, low-sulfur":
        # delete exchange
        exc.delete()
        # exc.save()

In [41]:
# Look at exhanges characterizing activity car_B
rows = []
for exc in car_B.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"petrol, low-sulfur",kilogram,0.0609059,"'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None)"
technosphere,"petrol, low-sulfur",kilogram,0.00116086,"'market for petrol, low-sulfur' (kilogram, CH, None)"


## Adding basalt quarry operation modified process

This processs is based on the following ecoinvent process:

- basalt quarry operation - RER

The new process is modified to set:
+ the basalt biosphere exchange amount to 0,
+ the technosphere limestone quarry infrastructure exchange amount to 0,
+ and multiply all the rest of the technosphere exchanges by 1,04. 

The name of the new process is set to

- basalt quarry operation (for crushed stone) - RER

In [42]:
# Delete the basalt quarry operation in case we ran this notebook multiple times
res = EI_DB.search("basalt quarry operation (for crushed stone)")
for r in res:
    print(r)
    r.delete()

'basalt quarry operation (for crushed stone)' (kilogram, RER, None)


In [43]:
# Search for basalt quarry operation in background database
q = Query()
filter_name = Filter("name", "is", "basalt quarry operation")
filter_location = Filter("location", "is", "RER")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    basalt = bd.get_activity(list(res).pop())
print(basalt)

'basalt quarry operation' (kilogram, RER, None)


### copy existing process

In [44]:
# Copy existing basalt process
basalt_cs = basalt.copy(name="basalt quarry operation (for crushed stone)")

In [45]:
# Verify that the new basalt process was recorded
EI_DB.search("basalt quarry operation")

['basalt quarry operation' (kilogram, RER, None),
 'basalt quarry operation' (kilogram, RoW, None),
 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)]

In [46]:
# Look at biosphere exhanges characterizing activity basalt_cs
rows = []
for exc in basalt_cs.biosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

biosphere,"Particulate Matter, < 2.5 um",kilogram,8e-06,"'Particulate Matter, < 2.5 um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,"Particulate Matter, > 10 um",kilogram,0.000112,"'Particulate Matter, > 10 um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,"Particulate Matter, > 2.5 um and < 10um",kilogram,4e-05,"'Particulate Matter, > 2.5 um and < 10um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,Water,cubic meter,1.68e-06,"'Water' (cubic meter, None, ('air',))"
biosphere,Water,cubic meter,6.72e-06,"'Water' (cubic meter, None, ('water',))"
biosphere,Basalt,kilogram,1,"'Basalt' (kilogram, None, ('natural resource', 'in ground'))"
biosphere,"Occupation, mineral extraction site",square meter-year,0.00798,"'Occupation, mineral extraction site' (square meter-year, None, ('natural resource', 'land'))"
biosphere,"Transformation, from forest, unspecified",square meter,0.000798,"'Transformation, from forest, unspecified' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, from mineral extraction site",square meter,0.000798,"'Transformation, from mineral extraction site' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, to mineral extraction site",square meter,0.000798,"'Transformation, to mineral extraction site' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, to unspecified",square meter,0.000798,"'Transformation, to unspecified' (square meter, None, ('natural resource', 'land'))"


In [47]:
# detail look to biosphere exchanges of interest
[
    exc for exc in basalt_cs.biosphere() if exc.get("name", "") == "Basalt"
].pop().as_dict()  # Basalt

{'flow': 'ac3a8914-35f0-4c34-a956-f26b3a053e4a',
 'type': 'biosphere',
 'name': 'Basalt',
 'classifications': {'CPC': []},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'water content': {'amount': 0.0, 'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'unit': 'kilogram',
 'amount': 1.0,
 'pedigree': {'reliability': 3,
  'completeness': 4,
  'temporal correlation': 5,
  'geographical correlation': 1,
  'further technological correlation': 3},
 'uncertainty type': 2,
 'loc': 0.0,
 'scale': 0.24899799195977465,
 'scale without pedigree': 0.1,
 'input': ('biosphere3', 'ac3a8914-35f0-4c34-a956-f26b3a053e4a'),
 'output'

In [48]:
# Iterate over biosphere exchanges in basalt_cs to change the amount of Basalt exchange to 0
for exc in basalt_cs.biosphere():
    if exc["name"] == "Basalt":
        # Update the amount in the exchange
        exc["amount"] = 0
        exc.save()

### Modify Basalt flow to amount = 0

In [49]:
# verify new exchange value for Basalt in basalt_cs
rows = []
for exc in basalt_cs.biosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

biosphere,"Particulate Matter, < 2.5 um",kilogram,8e-06,"'Particulate Matter, < 2.5 um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,"Particulate Matter, > 10 um",kilogram,0.000112,"'Particulate Matter, > 10 um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,"Particulate Matter, > 2.5 um and < 10um",kilogram,4e-05,"'Particulate Matter, > 2.5 um and < 10um' (kilogram, None, ('air', 'non-urban air or from high stacks'))"
biosphere,Water,cubic meter,1.68e-06,"'Water' (cubic meter, None, ('air',))"
biosphere,Water,cubic meter,6.72e-06,"'Water' (cubic meter, None, ('water',))"
biosphere,Basalt,kilogram,0,"'Basalt' (kilogram, None, ('natural resource', 'in ground'))"
biosphere,"Occupation, mineral extraction site",square meter-year,0.00798,"'Occupation, mineral extraction site' (square meter-year, None, ('natural resource', 'land'))"
biosphere,"Transformation, from forest, unspecified",square meter,0.000798,"'Transformation, from forest, unspecified' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, from mineral extraction site",square meter,0.000798,"'Transformation, from mineral extraction site' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, to mineral extraction site",square meter,0.000798,"'Transformation, to mineral extraction site' (square meter, None, ('natural resource', 'land'))"
biosphere,"Transformation, to unspecified",square meter,0.000798,"'Transformation, to unspecified' (square meter, None, ('natural resource', 'land'))"


In [50]:
# Look at technosphere exhanges characterizing activity basalt_cs
rows = []
for exc in basalt_cs.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"alkyd paint, white, without solvent, in 60% solution state",kilogram,3e-06,"'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None)"
technosphere,blasting,kilogram,7.73e-05,"'blasting' (kilogram, RER, None)"
technosphere,"chemical, organic",kilogram,3.6e-07,"'market for chemical, organic' (kilogram, GLO, None)"
technosphere,"diesel, burned in building machine",megajoule,0.0414,"'market for diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00621,"'market group for electricity, medium voltage' (kilowatt hour, RER, None)"
technosphere,"heat, district or industrial, other than natural gas",megajoule,0.00029735,"'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None)"
technosphere,limestone quarry infrastructure,unit,8e-11,"'market for limestone quarry infrastructure' (unit, GLO, None)"
technosphere,lubricating oil,kilogram,5.4e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,municipal solid waste,kilogram,-6.14e-05,"'market group for municipal solid waste' (kilogram, RER, None)"
technosphere,"transport, passenger car",kilometer,1.08e-05,"'market for transport, passenger car' (kilometer, RER, None)"
technosphere,waste graphical paper,kilogram,-5.2e-07,"'market group for waste graphical paper' (kilogram, RER, None)"


In [51]:
# detail look to technosphere exchange of interest
[
    exc
    for exc in basalt_cs.technosphere()
    if exc.get("name", "") == "limestone quarry infrastructure"
].pop().as_dict()  # limestone quarry infrastructure

{'flow': '1e278342-7e3b-426d-b000-58eab8104573',
 'type': 'technosphere',
 'name': 'limestone quarry infrastructure',
 'classifications': {'CPC': ['53269: Other constructions for manufacturing']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'}},
 'activity': 'e6a83983-3995-5ac3-9de9-d486f22896f9',
 'unit': 'unit',
 'comment': 'EcoSpold01Location=CH',
 'amount': 8e-11,
 'pedigree': {'reliability': 5,
  'completeness': 4,
  'temporal correlation': 5,
  'geographical correlation': 1,
  'further technological correlation': 3},
 'uncertainty type': 2,
 'loc': -23.248994481254666,
 'scale': 0.6557438524302001,
 'scale without pedigree': 0.5830951894845301,
 'input': ('ecoinvent-3.9.1-cutoff', '8333c52e7f25a01b4e063b97466be74c'),
 'output': ('ecoinvent-3.9.1-cutoff', '66dc54f1a59a4ca88cfadd4f461b8a73')}

In [52]:
# Iterate over technosphere exchanges in basalt_cs to change the amount of limestone quarry operation exchange to 0
for exc in basalt_cs.technosphere():
    if exc["name"] == "limestone quarry infrastructure":
        # Update the amount in the exchange
        exc["amount"] = 0
        exc.save()

In [53]:
# verify new exchange value for limestone quarry operation in basalt_cs
rows = []
for exc in basalt_cs.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"alkyd paint, white, without solvent, in 60% solution state",kilogram,3e-06,"'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None)"
technosphere,blasting,kilogram,7.73e-05,"'blasting' (kilogram, RER, None)"
technosphere,"chemical, organic",kilogram,3.6e-07,"'market for chemical, organic' (kilogram, GLO, None)"
technosphere,"diesel, burned in building machine",megajoule,0.0414,"'market for diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00621,"'market group for electricity, medium voltage' (kilowatt hour, RER, None)"
technosphere,"heat, district or industrial, other than natural gas",megajoule,0.00029735,"'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None)"
technosphere,limestone quarry infrastructure,unit,0,"'market for limestone quarry infrastructure' (unit, GLO, None)"
technosphere,lubricating oil,kilogram,5.4e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,municipal solid waste,kilogram,-6.14e-05,"'market group for municipal solid waste' (kilogram, RER, None)"
technosphere,"transport, passenger car",kilometer,1.08e-05,"'market for transport, passenger car' (kilometer, RER, None)"
technosphere,waste graphical paper,kilogram,-5.2e-07,"'market group for waste graphical paper' (kilogram, RER, None)"


In [54]:
# Iterate over technosphere exchanges in basalt_cs to multiply all technosphere exchanges time 1,04
for exc in basalt_cs.technosphere():
    # Update the amount in the exchange
    exc["amount"] = (exc["amount"]) * 1.04
    exc.save()

In [55]:
# verify new exchange technosphere values for basalt_cs
rows = []
for exc in basalt_cs.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,"alkyd paint, white, without solvent, in 60% solution state",kilogram,3.12e-06,"'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None)"
technosphere,blasting,kilogram,8.0392e-05,"'blasting' (kilogram, RER, None)"
technosphere,"chemical, organic",kilogram,3.744e-07,"'market for chemical, organic' (kilogram, GLO, None)"
technosphere,"diesel, burned in building machine",megajoule,0.043056,"'market for diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0064584,"'market group for electricity, medium voltage' (kilowatt hour, RER, None)"
technosphere,"heat, district or industrial, other than natural gas",megajoule,0.000309244,"'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None)"
technosphere,limestone quarry infrastructure,unit,0,"'market for limestone quarry infrastructure' (unit, GLO, None)"
technosphere,lubricating oil,kilogram,5.616e-05,"'market for lubricating oil' (kilogram, RER, None)"
technosphere,municipal solid waste,kilogram,-6.3856e-05,"'market group for municipal solid waste' (kilogram, RER, None)"
technosphere,"transport, passenger car",kilometer,1.1232e-05,"'market for transport, passenger car' (kilometer, RER, None)"
technosphere,waste graphical paper,kilogram,-5.408e-07,"'market group for waste graphical paper' (kilogram, RER, None)"


## Adding gravel production modified process

This processs is based on the following ecoinvent process:

- gravel production, crushed - RoW

The new process is modified to only include the following exchanges:

| Name                                               | Exchange type         | 
|----------------------------------------------------|-----------------------|
| gravel, crushed                                    | production            |
| building, hall, steel construction                 | technosphere          |
| conveyor belt                                      | technosphere          | 
| Gravel                                             | biosphere             | 
| gravel/sand quarry infrastructure                  | technosphere          | 
| industrial machine, heavy, unspecified             | technosphere          | 
| recultivation, limestone mine                      | technosphere          | 
| steel, low-alloyed, hot rolled                     | technosphere          | 
| synthetic rubber                                   | technosphere          | 


The name of the new process is set to:

- gravel production, crushed (for crushed stone) - RoW

In [56]:
# Delete the gravel production, crushed (for crushed stone) in case we ran this notebook multiple times
res = EI_DB.search("gravel production, crushed (for crushed stone) RoW")
for r in res:
    if r.get("name", "") == "gravel production, crushed (for crushed stone)" and (
        r.get("location", "") == "RoW"
    ):
        print(r)
        r.delete()

'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)


In [57]:
# Search for gravel production, crushed in background database
EI_DB.search("gravel production, crushed RoW")

['gravel production, crushed' (kilogram, RoW, None),
 'market for gravel, crushed' (kilogram, RoW, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'limestone production, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, RoW, None),
 'market for limestone, crushed, for mill' (kilogram, RoW, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'market for anhydrite rock' (kilogram, RoW, None)]

In [58]:
# Copy existing gravel process
q = Query()
filter_name = Filter("name", "is", "gravel production, crushed")
filter_location = Filter("location", "is", "RoW")
q.add(filter_name)
q.add(filter_location)
res = q(eidb_data)
if len(res) == 1:
    gravel = bd.get_activity(list(res).pop())
    print(gravel)

gravel_cs = gravel.copy(name="gravel production, crushed (for crushed stone)")

'gravel production, crushed' (kilogram, RoW, None)


In [59]:
# Verify that the new gravel process was recorded
EI_DB.search("gravel production, crushed RoW")

['gravel production, crushed' (kilogram, RoW, None),
 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None),
 'market for gravel, crushed' (kilogram, RoW, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'limestone production, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, RoW, None),
 'market for limestone, crushed, for mill' (kilogram, RoW, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'market for anhydrite rock' (kilogram, RoW, None)]

In [60]:
# Look at exhanges characterizing activity gravel_cs
rows = []
for exc in gravel_cs.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"gravel, crushed",kilogram,1,"'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)"
technosphere,"building, hall, steel construction",square meter,2.85e-06,"'market for building, hall, steel construction' (square meter, GLO, None)"
technosphere,conveyor belt,meter,9.51e-08,"'market for conveyor belt' (meter, GLO, None)"
technosphere,"diesel, burned in building machine",megajoule,0.0143,"'market for diesel, burned in building machine' (megajoule, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,6.48436e-06,"'market for electricity, medium voltage' (kilowatt hour, PY, None)"
technosphere,"electricity, medium voltage",kilowatt hour,9.6133e-06,"'market for electricity, medium voltage' (kilowatt hour, EC, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.000332695,"'market group for electricity, medium voltage' (kilowatt hour, RME, None)"
technosphere,"electricity, medium voltage",kilowatt hour,4.56887e-06,"'market for electricity, medium voltage' (kilowatt hour, CR, None)"
technosphere,"electricity, medium voltage",kilowatt hour,7.53338e-06,"'market for electricity, medium voltage' (kilowatt hour, US-PR, None)"
technosphere,"electricity, medium voltage",kilowatt hour,3.57918e-06,"'market for electricity, medium voltage' (kilowatt hour, TT, None)"
technosphere,"electricity, medium voltage",kilowatt hour,8.91359e-05,"'market for electricity, medium voltage' (kilowatt hour, AU, None)"


In [61]:
# detail look to exchanges of interest
[
    exc for exc in gravel_cs.exchanges() if exc.get("name", "") == "gravel, crushed"
].pop().as_dict()  # gravel, crushed

{'flow': '37eceabd-b33f-4757-a4b9-5c51dee1710d',
 'type': 'production',
 'name': 'gravel, crushed',
 'classifications': {'CPC': ['15320: Pebbles, gravel, broken or crushed stone, macadam; granules, chippings and powder of stone']},
 'production volume': 3684834500000.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0,
   'comment': 'SiO2, ecoinvent v2.2 report 7, part I',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0,
   'comment': 'SiO2, ecoinvent v2.2 report 7, part I',
   'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'price': {'amount': 0.00446,
   'comment': 'World market prices are estimated to be equal to USA prices, which are given according to http://minerals.usgs.gov/minerals/pubs/commodity/sand_&_gravel_construction/myb1-2007-sandc.xls, table 1, downloaded 21 April 2009. NOTE: The crushing process

### Define new values for the exchanges

In [62]:
# create dictionary with exchanges to keep
gravel_cs_exc = [
    {"name": "gravel, crushed", "type": "production"},
    {"name": "building, hall, steel construction", "type": "technosphere"},
    {"name": "conveyor belt", "type": "technosphere"},
    {"name": "Gravel", "type": "biosphere"},
    {"name": "gravel/sand quarry infrastructure", "type": "technosphere"},
    {"name": "industrial machine, heavy, unspecified", "type": "technosphere"},
    {"name": "recultivation, limestone mine", "type": "technosphere"},
    {"name": "steel, low-alloyed, hot rolled", "type": "technosphere"},
    {"name": "synthetic rubber", "type": "technosphere"},
]

### Modify gravel_cs exchanges

In [63]:
# Iterate over exchanges in gravel_cs to delete exchanges that are not included in gravel_cs_exc
for exc in gravel_cs.exchanges():
    found = False
    for item in gravel_cs_exc:
        if exc["name"] == item["name"] and exc["type"] == item["type"]:
            found = True
            break
    if not found:
        exc.delete()
        # exc.save()

In [64]:
# verify new exchanges for gravel_cs
rows = []
for exc in gravel_cs.exchanges():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

production,"gravel, crushed",kilogram,1,"'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)"
technosphere,"building, hall, steel construction",square meter,2.85e-06,"'market for building, hall, steel construction' (square meter, GLO, None)"
technosphere,conveyor belt,meter,9.51e-08,"'market for conveyor belt' (meter, GLO, None)"
technosphere,gravel/sand quarry infrastructure,unit,4.75e-11,"'market for gravel/sand quarry infrastructure' (unit, GLO, None)"
technosphere,"industrial machine, heavy, unspecified",kilogram,9.51e-05,"'market for industrial machine, heavy, unspecified' (kilogram, RoW, None)"
technosphere,"recultivation, limestone mine",square meter,1.27e-06,"'market for recultivation, limestone mine' (square meter, GLO, None)"
technosphere,"steel, low-alloyed, hot rolled",kilogram,5.1e-05,"'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)"
technosphere,synthetic rubber,kilogram,4e-06,"'market for synthetic rubber' (kilogram, GLO, None)"
biosphere,Gravel,kilogram,1.04,"'Gravel' (kilogram, None, ('natural resource', 'in ground'))"


## Adding cellullose fibre production modified process

This processs is based on the following ecoinvent process:

- cellulose fibre production - RoW

The new process is modified to exclude the *borax, anhydrous, powder* and *boric acid, anhydrous, powder* technosphere exchanges.

The name of the new process is set to:

- cellulose fibre production (without borax and boric acid) - RoW

In [65]:
# Delete the cellulose fibre production (without borax and boric acid) in case we ran this notebook multiple times
res = EI_DB.search("cellulose fibre production (without borax and boric acid) RoW")
for r in res:
    if r.get(
        "name", ""
    ) == "cellulose fibre production (without borax and boric acid)" and (
        r.get("location", "") == "RoW"
    ):
        print(r)
        r.delete()

'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)


In [66]:
# Search for cellulose fibre production in background database
EI_DB.search("cellulose fibre production RoW")

['cellulose fibre production' (kilogram, RoW, None)]

### Copy existing cellulose fibre process

In [67]:
# Copy existing cellulose fibre process
cellulose = EI_DB.search("cellulose fibre production")[1]
q = Query()
filter_name = Filter("name", "is", "cellulose fibre production")
filter_location = Filter("location", "is", "RoW")
res = q(eidb_data)
if len(res) == 1:
    cellulose = bd.get_activity(list(res).pop())
print(cellulose)

'cellulose fibre production' (kilogram, RoW, None)


In [68]:
cellulose_mod = cellulose.copy(
    name="cellulose fibre production (without borax and boric acid)"
)

In [69]:
# Verify that the new cellulose fibre process was recorded
EI_DB.search("cellulose fibre production (without borax and boric acid)")

['cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)]

In [70]:
# Look at technosphere exhanges characterizing activity cellulose_mod
rows = []
for exc in cellulose_mod.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,aluminium hydroxide,kilogram,0.04,"'market for aluminium hydroxide' (kilogram, GLO, None)"
technosphere,"boric acid, anhydrous, powder",kilogram,0.04,"'market for boric acid, anhydrous, powder' (kilogram, GLO, None)"
technosphere,"building, hall, steel construction",square meter,4.24e-06,"'market for building, hall, steel construction' (square meter, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00119392,"'market for electricity, medium voltage' (kilowatt hour, AU, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0720916,"'market group for electricity, medium voltage' (kilowatt hour, RAS, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00785732,"'market group for electricity, medium voltage' (kilowatt hour, RLA, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0171548,"'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00534402,"'market for electricity, medium voltage' (kilowatt hour, RU, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0251811,"'market group for electricity, medium voltage' (kilowatt hour, RNA, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0039433,"'market group for electricity, medium voltage' (kilowatt hour, RAF, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.000233991,"'market for electricity, medium voltage' (kilowatt hour, NZ, None)"


In [71]:
# Iterate over exchanges in cellulose_mod to delete borax, anhydrous, powder and boric acid, anhydrous, powder technosphere exchanges
for exc in cellulose_mod.technosphere():
    if exc["name"] == "borax, anhydrous, powder":
        exc.delete()
        # exc.save()
    elif exc["name"] == "boric acid, anhydrous, powder":
        exc.delete()
        # exc.save()

In [72]:
# verify new technosphere exchanges for cellulose_mod
rows = []
for exc in cellulose_mod.technosphere():
    exc_row = {
        field: value
        for field, value in exc.as_dict().items()
        if field in ["amount", "unit", "name", "type"]
    }
    exchange_activity = bd.get_activity(exc["input"])
    exc_row["activity"] = str(exchange_activity)
    rows.append(exc_row)
tabulate(rows, tablefmt="html")

technosphere,aluminium hydroxide,kilogram,0.04,"'market for aluminium hydroxide' (kilogram, GLO, None)"
technosphere,"building, hall, steel construction",square meter,4.24e-06,"'market for building, hall, steel construction' (square meter, GLO, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00119392,"'market for electricity, medium voltage' (kilowatt hour, AU, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0720916,"'market group for electricity, medium voltage' (kilowatt hour, RAS, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00785732,"'market group for electricity, medium voltage' (kilowatt hour, RLA, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0171548,"'market group for electricity, medium voltage' (kilowatt hour, Europe without Switzerland, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.00534402,"'market for electricity, medium voltage' (kilowatt hour, RU, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0251811,"'market group for electricity, medium voltage' (kilowatt hour, RNA, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.0039433,"'market group for electricity, medium voltage' (kilowatt hour, RAF, None)"
technosphere,"electricity, medium voltage",kilowatt hour,0.000233991,"'market for electricity, medium voltage' (kilowatt hour, NZ, None)"
technosphere,"heat, district or industrial, natural gas",megajoule,0.0462426,"'market for heat, district or industrial, natural gas' (megajoule, RoW, None)"


## Final verification of the modified background procs

In [73]:
new_proc_names = [
    "A5, diesel, burned in building machine",
    "C1, diesel, burned in building machine",
    "B, transport, freight, lorry 7.5-16 metric ton, EURO5",
    "B, transport, passenger car, medium size, petrol, EURO 5",
    "basalt quarry operation (for crushed stone)",
    "gravel production, crushed (for crushed stone)",
    "cellulose fibre production (without borax and boric acid)",
]

In [74]:
eidb_data = EI_DB.load()

In [75]:
new_procs_found = []
for new_proc_name in new_proc_names:
    q = Query()
    filter_name = Filter("name", "is", new_proc_name)
    q.add(filter_name)
    res = q(eidb_data)
    if len(res) == 1:
        print(f"✅ Found: {bd.get_activity(list(res).pop())}")
        new_procs_found.append(res)
    else:
        print(f"❓ Multiple results ({len(res)}) found for {new_proc_name} - {res}")
assert len(new_procs_found) == len(new_proc_names)

✅ Found: 'A5, diesel, burned in building machine' (megajoule, GLO, None)
✅ Found: 'C1, diesel, burned in building machine' (megajoule, GLO, None)
✅ Found: 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)
✅ Found: 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)
✅ Found: 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)
✅ Found: 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)
✅ Found: 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)
